Imports & installs.

In [ ]:
!pip install py7zr

In [ ]:
################################### IMPORTS ###################################
###
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt

###
from PIL import Image

###
import torch
# neural network modules, nn.Linear, nn.Conv2d, loss functions
import torch.nn as nn
# activation functions
import torch.nn.functional as F
# optimizers
import torch.optim as optim
# transformations 
import torchvision.transforms as transforms

from torchvision import models
import torchvision
from torch.utils.data import DataLoader, Dataset, Subset

###
from sklearn.model_selection import train_test_split

# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

###
import warnings
warnings.filterwarnings("ignore")

###
from py7zr import unpack_7zarchive
import shutil
shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)

In [ ]:
################################# HYPERPARAMS #################################

num_epochs = 3
batch_size = 64
lr = 0.01

Load data.

In [ ]:
############################# CUSTOM DATASET CLASS #############################

class CifarLoader(Dataset):
  def __init__(self, root_dir, csv_file = None, transform = None):

    # images directory 
    self.root_dir = root_dir
    # transformations if any
    self.transform = transform

    if csv_file:
      # read csv file
      self.df = pd.read_csv(csv_file)

      # {class: lbl}
      self.annot_dict = {}

      for i, lbl in enumerate(self.df['label'].unique()):
        self.annot_dict[lbl] = i

      # {lbl: class}
      self.annot_dict_reversed = {v:k for k,v in self.annot_dict.items()}

      # add a column with decoded labels
      self.df['encoded'] = self.df['label'].map(self.annot_dict)

    else: 
      # if no csv file provided -> create a df, fill labels with 0's
      self.df = pd.DataFrame({
          'id': list(range(1, len(os.listdir(self.root_dir)) + 1)),
          'label': np.zeros(len(os.listdir(self.root_dir))),
          'encoded': np.zeros(len(os.listdir(self.root_dir)))
      })


  def __len__(self):
    # num of records in the df
    return len(self.df)

  def __getitem__(self, index):
    # path to the image
    img_path = os.path.join(self.root_dir, str(self.df.iloc[index, 0]) + '.png')
    # read image
    image = np.array(Image.open(img_path).convert('RGB'))
    # get label
    y_label = torch.tensor(int(self.df.iloc[index, 2]))

    # apply transformations
    if self.transform:
      image = self.transform(image)
    
    return (image, y_label)


In [ ]:
# transform tensors to normalized range form [0, 1] to [-1, 1]
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.5, 0.5, 0.5])

tr = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),        
        transforms.Normalize(mean, std)
    ])

In [ ]:
# unzip train and test data
shutil.unpack_archive('../input/cifar-10/train.7z')
shutil.unpack_archive('../input/cifar-10/test.7z')

In [ ]:
dataset = CifarLoader(csv_file = '../input/cifar-10/trainLabels.csv', 
                      root_dir = './train',
                      transform = tr)

print(dataset.df.head())

In [ ]:
############################### SPLIT THE DATA #################################

indexes = list(range(len(dataset)))

train_indexes, val_indexes = train_test_split(indexes, test_size=0.2)

data = {}
data['train'] = Subset(dataset, train_indexes)
data['val'] = Subset(dataset, val_indexes)

dataloader_train = DataLoader(dataset = data['train'], batch_size = batch_size, shuffle=False)
dataloader_val = DataLoader(dataset = data['val'], batch_size = batch_size, shuffle=False)

In [ ]:
print(len(train_indexes), len(val_indexes))

In [ ]:
################################ SHOW IMAGES ##################################

def images_show(images, lbls=None):
  # show only 4 first images from each batch
  for i in range(4):
    plt.subplot(1, 4, i + 1)
    # convert to numpy
    img = images[i].numpy().transpose((1, 2, 0))
    # unnormalize
    img = img * std + mean 
    img = np.clip(img, a_min=0, a_max=1)
    plt.imshow(img)
 
    if lbls != None:
      # decode labels using reversed annotation dictionary
      plt.title(dataset.annot_dict_reversed[lbls[i].item()])
    plt.axis('off')
  plt.tight_layout(pad=0.5)
  plt.show()

In [ ]:
dataiter = iter(dataloader_train)
images, labels = dataiter.next()

print(f'Shape: {images.shape}')
print(f'Max: {torch.max(images[0]):.2f}')
print(f'Min: {torch.min(images[0]):.2f}')

for i in range(3):
  images, labels = dataiter.next()
  images_show(images, labels)

Load pretrained DenseNet-121.

In [ ]:
################################ FINETUNING ##################################

# Load a pretrained model and reset final fully connected layer

# load a pretrained model 
model = models.densenet121(pretrained=True)

# reset final fully connected layer
num_ftrs = model.classifier.in_features

model.classifier = nn.Sequential(
                        nn.Linear(num_ftrs, 256),  
                        nn.ReLU(), 
                        nn.Dropout(0.3),
                        nn.Linear(256, 10))

# copy weights for futher retraining on full train dataset
model_wts = copy.deepcopy(model.state_dict())

# move model to a device
model = model.to(device)

# loss
criterion = nn.CrossEntropyLoss()

# all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=lr)

In [ ]:
################################ TRAINING FUNC ################################

def train(model, criterion, optimizer, num_epochs, 
          dataloader_train, dataloader_val = None):

  history = {
        'loss': [],
        'accuracy': [],
        'val_loss': [],
        'val_accuracy': [],
        'best train acc': (0, 0)
        }

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0


  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch + 1, num_epochs))

    # set model to the training mode
    model.train()

    n_samples = 0
    correct_train = 0
    epoch_loss_train = 0
    epoch_loss_val = 0


    for i, (images, labels) in enumerate(dataloader_train):
      # origin shape: [4, 3, 32, 32] = 4, 3, 1024
    
      images = images.to(device)
      labels = labels.to(device)

      # forward pass (pred)
      pred = model(images)

      # loss
      loss = criterion(pred, labels)

      # mean loss * num samples in batch
      epoch_loss_train += labels.shape[0] * loss.item()

      # empty gradients
      optimizer.zero_grad()

      # gradient (backpropagation)
      loss.backward()

      # update weights
      optimizer.step()

      # values, indexes
      value, predicted = torch.max(pred, 1)

      # += batch_size
      n_samples += labels.shape[0]
      # num of correctly predicted in this batch
      correct_train += (predicted==labels).sum().item()

    # train acc per epoch
    train_acc = 100 * correct_train / n_samples
    # train loss per epoch
    epoch_loss_train = epoch_loss_train / n_samples

    print(f'Train accuracy: {train_acc:.2f}, loss: {epoch_loss_train:.2f}')
    #print(f'Best accuracy: {best_acc:.2f}')

    history['accuracy'].append(train_acc)
    history['loss'].append(epoch_loss_train)

    # find best accuracy on training data
    if train_acc > best_acc:

      #print('*New best accuracy*')
      best_acc = train_acc

      # copy current model weights
      best_model_wts = copy.deepcopy(model.state_dict())
      history['best train acc'] = (epoch, best_acc)



    if dataloader_val:

      # evaluation mode
      model.eval()

      with torch.no_grad():
        correct_val = 0
        n_samples = 0

        for images, labels in dataloader_val: 
          images = images.to(device)
          labels = labels.to(device)

          outputs = model(images)

          loss = criterion(outputs, labels)
          epoch_loss_val += labels.shape[0] * loss.item()

          # value, index
          _, pred = torch.max(outputs, 1)
          n_samples += labels.shape[0]
          correct_val += (pred==labels).sum().item()

        # calculate validation accuracy and loss for each epoch
        val_acc = 100 * correct_val / len(data['val'])
        epoch_loss_val = epoch_loss_val / len(data['val'])

        print(f'Val accuracy: {val_acc:.2f}, loss: {epoch_loss_val:.2f}')

        history['val_accuracy'].append(val_acc)
        history['val_loss'].append(epoch_loss_val)
      
    print('=' * 80)


  print('Best train acc: {:2f}'.format(best_acc))

  # load best weights
  model.load_state_dict(best_model_wts)

  return model, history

In [ ]:
model, history = train(model, criterion, optimizer, num_epochs, 
                       dataloader_train, dataloader_val)

In [ ]:
def plot_history(history):
   
    loss = history['loss']
    accuracy = history['accuracy']
    val_loss = history['val_loss']
    val_accuracy = history['val_accuracy']
    x = range(len(loss))

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, accuracy, label='Training acc', color='#03045e', linewidth=2)
    if len(val_loss) != 0:
      plt.plot(x, val_accuracy, label='Validation acc', color='#48cae4', linewidth=2)
    plt.plot(history['best train acc'][0], 
             history['best train acc'][1], 
             'bo', label='Best train acc', markersize=7, color='black')
    plt.title('Accuracy')
    plt.grid(True)
    plt.legend()
    
    
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, label='Training loss', color='#03045e', linewidth=2)
    if len(val_loss) != 0:
      plt.plot(x, val_loss, label='Validation loss', color='#48cae4', linewidth=2)
    plt.title('Loss')
    plt.grid(True)
    plt.legend()

In [ ]:
plot_history(history)

In [ ]:
################################## EVALUATE ##################################
def evaluate(model, dataloader):

  with torch.no_grad():
    model.eval()
    n_correct = 0
    n_samples = 0

    for images, labels in dataloader: 
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)

      # value, index
      v, pred = torch.max(outputs, 1)

      n_samples += labels.shape[0]
      n_correct += (pred==labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(acc)

In [ ]:
evaluate(model, dataloader_val)

Retrain model on full training set.

In [ ]:
############################## COMPLETE DATASET ###############################

dataloader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle=False)

In [ ]:
dataset_test = CifarLoader(root_dir = './test',
                      transform = tr)

dataloader_test = DataLoader(dataset = dataset_test, 
                             batch_size = batch_size, shuffle=False)

In [ ]:
################################# RESET MODEL ##################################

model.load_state_dict(model_wts)

# loss
criterion = nn.CrossEntropyLoss()

# all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=lr)

In [ ]:
model, history = train(model, criterion, optimizer, 3, dataloader)

In [ ]:
print(len(dataset_test))

In [ ]:
################################### PREDICT ####################################

predictions = np.array([])

with torch.no_grad():
    model.eval()

    for images, _ in dataloader_test:
      images = images.to(device)

      outputs = model(images)

      # value, index
      v, pred = torch.max(outputs, 1)

      pred = pred.cpu().numpy()

      predictions = np.concatenate((predictions, pred), axis = None)

In [ ]:
dataset_test.df['encoded'] = predictions.astype(int)

In [ ]:
dataset_test.df['label'] = dataset_test.df['encoded'].map(dataset.annot_dict_reversed)

In [ ]:
dataset_test.df.head()

In [ ]:
dataiter = iter(dataloader_test)
images, labels = dataiter.next()

for i in range(3):
  images, labels = dataiter.next()
  images_show(images, labels)

Submission file:

In [ ]:
dataset_test.df.drop(columns ='encoded', inplace=True)
dataset_test.df.to_csv('submission.csv', index=False)

In [ ]:
%rm -rf ./train
%rm -rf ./test